## Table des matières

- [Table population](#population)
- [Table dispo_alim](#dispo_alim)
- [Table equilibre_prod](#equilibre_prod)
- [Table sous_nutrition](#sous_nutrition)
- [Export des tables en fichiers csv](#export)

In [1]:
import numpy as np
import pandas as pd

pop = pd.read_csv("fr_population.csv")
animaux = pd.read_csv("fr_animaux.csv")
vegetaux = pd.read_csv("fr_vegetaux.csv")
sousalimentation = pd.read_csv("fr_sousalimentation.csv")

# Suppression de l'élément "Chine" avec le code zone

pop = pop[pop["Code zone"]!=351]
animaux = animaux[animaux["Code zone"]!=351]
sousalimentation = sousalimentation[sousalimentation["Code zone"]!=351]
vegetaux = vegetaux[vegetaux["Code zone"]!=351]

<a id="population"></a>
#### <span style="background-color: #FFFF00">Table population</span>

In [2]:
population = pop.loc[ : ,('Zone','Code zone','Année','Valeur')]

population.Valeur *= 1000

population = population.rename(columns={"Zone":"pays", 
                                        "Code zone":"code_pays", 
                                        "Année":"annee", 
                                        "Valeur":"population"})

# Lors de l'importation de la table population dans SQLite,
# la présence d'une virgule pour les pays 
# "Chine, Taiwan province de" et "Chine, continentale"
# a créé un décalage des données de cette ligne sur le fichier .csv 
# rendant inexploitable la table. 
# Il faut donc renommer cette ligne afin de supprimer la virgule.

population = population.replace("Chine, Taiwan Province de", "Taiwan")
population = population.replace("Chine, continentale", "Chine continentale")

population.head(3)

,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000


<a id="dispo_alim"></a>
#### <span style="background-color: #FFFF00">Table dispo_alim</span>

In [3]:
animaux["origin"]= "animal"
vegetaux["origin"]= "végétal"

dispo_alim = pd.concat([animaux,vegetaux])
dispo_alim = dispo_alim.pivot_table(index =['Zone','Code zone','Année','Produit','Code Produit','origin'],
                            columns='Élément', values='Valeur', aggfunc = sum)
dispo_alim = dispo_alim.loc[ : ,('Disponibilité alimentaire en quantité (kg/personne/an)',
                                 'Disponibilité alimentaire (Kcal/personne/jour)',
                                 'Disponibilité de protéines en quantité (g/personne/jour)',
                                 'Disponibilité de matière grasse en quantité (g/personne/jour)'
                                )]
dispo_alim = dispo_alim.reset_index()


dispo_alim = dispo_alim.rename(columns={'Zone':'pays', 
                           'Code zone': 'code_pays', 
                           'Année':'année',
                           'Produit':'produit',
                           'Code Produit':'code_produit',
                           'Disponibilité alimentaire en quantité (kg/personne/an)':'dispo_alim_tonnes',
                           'Disponibilité alimentaire (Kcal/personne/jour)':'dispo_alim_kcal_p_j',
                           'Disponibilité de protéines en quantité (g/personne/jour)':'dispo_alim_prot',
                           'Disponibilité de matière grasse en quantité (g/personne/jour)':'dispo_mat_gr'  
                                       })

dispo_alim = pd.merge(dispo_alim,population)
dispo_alim['dispo_alim_tonnes'] = dispo_alim['dispo_alim_tonnes']*dispo_alim['population']/1000
dispo_alim = dispo_alim.drop(columns=['annee','population'])
dispo_alim = dispo_alim.replace("Chine, Taiwan Province de", "Taiwan")
dispo_alim = dispo_alim.replace("Chine, continentale", "Chine continentale")
dispo_alim = dispo_alim.drop_duplicates()
dispo_alim = dispo_alim.dropna()

dispo_alim.head(3)

,pays,code_pays,année,produit,code_produit,origin,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_alim_prot,dispo_mat_gr
0,Afghanistan,2,2013,Abats Comestible,2736,animal,52549.44,5.0,0.77,0.20
1,Afghanistan,2,2013,"Agrumes, Autres",2614,végétal,39412.08,1.0,0.02,0.01
2,Afghanistan,2,2013,Aliments pour enfants,2680,végétal,1833.12,1.0,0.03,0.01


<a id="equilibre_prod"></a>
#### <span style="background-color: #FFFF00">Table equilibre_prod</span>

In [4]:
equilibre_prod = pd.concat([animaux,vegetaux])
equilibre_prod.Valeur *= 1000
equilibre_prod = equilibre_prod.pivot_table(index =['Zone','Code zone','Année','Produit','Code Produit'],
                            columns='Élément', values='Valeur', aggfunc = sum)
equilibre_prod = equilibre_prod.loc[ : ,('Disponibilité intérieure',
                                         'Aliments pour animaux',
                                         'Semences',
                                         'Pertes',
                                         'Traitement',
                                         'Nourriture',
                                         'Autres utilisations (non alimentaire)'
                                         )]
equilibre_prod = equilibre_prod.reset_index()


equilibre_prod = equilibre_prod.rename(columns={'Zone':'pays', 
                                                'Code zone': 'code_pays', 
                                                'Année':'année',
                                                'Produit':'produit',
                                                'Code Produit':'code_produit',
                                                'Disponibilité intérieure':'dispo_int',
                                                'Aliments pour animaux':'alim_ani',
                                                'Semences':'semences',
                                                'Pertes':'pertes',
                                                'Traitement':'transfo',
                                                'Nourriture':'nourriture',
                                                'Autres utilisations (non alimentaire)':'autres_utilisations',
                                       })

equilibre_prod = equilibre_prod.replace("Chine, Taiwan Province de", "Taiwan")
equilibre_prod = equilibre_prod.replace("Chine, continentale", "Chine continentale")
equilibre_prod = equilibre_prod.drop_duplicates()
equilibre_prod = equilibre_prod.dropna()

equilibre_prod.head(3)

Élément,pays,code_pays,année,produit,code_produit,dispo_int,alim_ani,semences,pertes,transfo,nourriture,autres_utilisations
111,Afrique du Sud,202,2013,Maïs,2514,10480000.0,4715000.0,33000.0,405000.0,43000.0,5283000.0,0.0
128,Afrique du Sud,202,2013,"Plantes Oleiferes, Autre",2570,16000.0,0.0,0.0,0.0,14000.0,0.0,1000.0
135,Afrique du Sud,202,2013,Pommes de Terre,2531,2126000.0,217000.0,79000.0,109000.0,0.0,1618000.0,103000.0


<a id="sous_nutrition"></a>
#### <span style="background-color: #FFFF00">Table sous_nutrition</span>

In [5]:
sous_nutrition = sousalimentation.loc[ : ,("Zone", "Code zone", "Année", "Valeur")]
sous_nutrition = sous_nutrition[sous_nutrition.Année== "2012-2014"]
sous_nutrition.Valeur = pd.to_numeric(sous_nutrition.Valeur, errors='coerce')
sous_nutrition.Valeur *= 1000000
sous_nutrition = sous_nutrition.dropna()
sous_nutrition = sous_nutrition.rename(columns={"Zone":"pays",
                                                "Code zone":"code_pays",
                                                "Année":"année",
                                                "Valeur": "nb_personnes"})
sous_nutrition = sous_nutrition.replace({"Chine, Taiwan Province de":"Taiwan",
                                         "Chine, continentale": "Chine continentale"})
sous_nutrition = sous_nutrition.replace("Chine, continentale", "Chine continentale")

sous_nutrition.head(3)

,pays,code_pays,année,nb_personnes
0,Afghanistan,2,2012-2014,7900000.0
5,Afrique du Sud,202,2012-2014,2600000.0
10,Albanie,3,2012-2014,200000.0


<a id="export"></a>
#### <span style="background-color: #FFFF00">Export des tables en fichiers csv</span>

In [6]:
# Rajout d'un espace au début de chaque élément de la colone "Pays" de chaque tables

population['pays'] = ' ' + population['pays']
dispo_alim['pays'] = ' ' + dispo_alim['pays']
equilibre_prod['pays'] = ' ' + equilibre_prod['pays']
sous_nutrition['pays'] = ' ' + sous_nutrition['pays']

population.to_csv("population.csv", sep=',', float_format='%.2f',index=False, line_terminator='\n')
dispo_alim.to_csv("dispo_alim.csv", sep=',', float_format='%.2f',index=False, line_terminator='\n')
equilibre_prod.to_csv("equilibre_prod.csv", sep=',', float_format='%.2f',index=False, line_terminator='\n')
sous_nutrition.to_csv("sous_nutrition.csv", sep=',', float_format='%.2f',index=False, line_terminator='\n')